In [ ]:
!git clone https://github.com/LukaMelinc/NMRV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary libraries
!pip install albumentations segmentation-models-pytorch ultralytics -q

In [ ]:
from ultralytics import YOLO
import torch
import torch.optim as optim

print("loading YOLOv11")
# Load the pre-trained YOLOv11 model
model = YOLO('yolov8m.pt')

print("loading YOLOv11 done")
# Set the model to training mode
model.trainable = True

"""for name, param in model.model.named_parameters():
    if 'head' not in name:
        param.requires_grad = False"""

print("training model")

# Define the optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Early stopping parameters
patience = 10
best_val_loss = float('inf')
epochs_without_improvement = 0

# Train the model with your custom dataset and data augmentation
model.train(
    data='/content/NMRV/Seminar/data_RGB.yaml',
    epochs=50,
    imgsz=640,
    batch=32,
    device="cuda",  # Use GPU if available, set to 'cpu' if no GPU
    augment=True,  # Enable data augmentation
    hsv_h=0.015,  # Hue augmentation
    hsv_s=0.7,    # Saturation augmentation
    hsv_v=0.4,    # Value augmentation
    degrees=0.0,  # Rotation degrees
    translate=0.1,  # Translation fraction
    scale=0.5,    # Scaling fraction
    shear=0.0,    # Shear fraction
    perspective=0.0,  # Perspective fraction
    flipud=0.0,   # Vertical flip probability
    fliplr=0.5,   # Horizontal flip probability
    mosaic=1.0,   # Mosaic data augmentation probability
    mixup=0.0     # Mixup data augmentation probability
)

# Save the trained model weights
model_save_path = '/content/drive/MyDrive/models'

# Training loop with early stopping and learning rate scheduler
for epoch in range(50):
    # Train the model for one epoch
    model.train(data='/content/NMRV/Seminar/data_RGB.yaml', epochs=1, imgsz=640, batch=32, device="cuda")

    # Validate the model
    val_loss = model.val(data='/content/NMRV/Seminar/data_RGB.yaml', imgsz=640, batch=32, device="cuda")

    # Update the learning rate scheduler
    scheduler.step(val_loss)

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        model.save(model_save_path)
        print(f"Model saved to {model_save_path}")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered")
            break

print(f"Training completed. Best validation loss: {best_val_loss}")
